In [0]:

import delta
import sys

sys.path.insert(0,"../lib/")

import utils
import ingestors

In [0]:
catalog = "bronze"
schema = "f1_world"
#tablename = dbutils.widgets.get("tableName")
tablename = "races"
path = f"/Volumes/raw/{schema}/{tablename}/"


In [0]:
ingestor = ingestors.Ingestor(spark,catalog=catalog, schemaname=schema, tablename=tablename, data_format='csv', target_race_id=target_race_id)
if not utils.table_exists(spark, catalog, schema, tablename):
  print(f"Criando tabela {tablename}")
  df = ingestor.load(path)
  ingestor.save_batch(df)

else:
  print(f"Tabela {tablename} ja existe")
                                        
